In [ ]:
%pip install torch numpy pandas fastai

PADDY DISEASE CLASSIFICATION

In [ ]:

from pathlib import Path
from fastai.vision.all import *

path = Path("/notebooks/paddy-disease-classification")
path.ls()

In [ ]:
trn_path = path/'train_images'
files = get_image_files(trn_path)

In [ ]:
img = PILImage.create(files[0])
print(img.size)
img.to_thumb(128)

In [ ]:
from fastcore.parallel import *

def f(o): return PILImage.create(o).size
sizes = parallel(f,files,n_workers=12)
pd.Series(sizes).value_counts()

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42, item_tfms=Resize(480,method='squish'),batch_tfms=aug_transforms(size=128,min_scale=0.75))

In [ ]:
dls.show_batch(max_n=6)

In [ ]:
%pip install --upgrade timm  huggingface_hub

In [ ]:
from huggingface_hub import *
learn = vision_learner(dls, 'resnet26d',metrics=error_rate,path='.').to_fp16()

In [ ]:
learn.lr_find(suggest_funcs=(valley,slide))

In [ ]:
learn.fine_tune(3,0.01)

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
ss

In [ ]:
tst_files = get_image_files(path/'test_images').sorted()
tst_dl = dls.test_dl(tst_files)

In [ ]:
probs,_,idxs= learn.get_preds(dl=tst_dl,with_decoded=True)
idxs

In [ ]:
dls.vocab

In [ ]:
mapping = dict(enumerate(dls.vocab))
results = pd.Series(idxs.numpy(),name="idxs").map(mapping)
results

In [ ]:
ss['label'] = results
ss

In [ ]:
ss.to_csv('submission.csv',index=False)

In [ ]:
iskaggle= False
if not iskaggle:
    from kaggle import api
    api.competition_submit('submission.csv','initial rn26d','paddy-disease-classification')

In [ ]:
trn_path = Path('sml')

resize_images(path/'train_images'/'bacterial_panicle_blight', dest=trn_path,max_size=192,recurse=True)

In [ ]:
dls= ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42, item_tfms=Resize(256,method='squish'),batch_tfms=aug_transforms(size=128,min_scale=0.75))

In [ ]:
trn_path = path/'train_images'/'bacterial_panicle_blight'
def train(arch,item,batch,epochs = 5):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42, item_tfms=item,batch_tfms=batch)
    learn = vision_learner(dls, arch,metrics=error_rate,path='.').to_fp16()
    learn.fine_tune(epochs,0.01)
    return learn

learn = train('resnet26d',Resize(192,method='squish'),aug_transforms(size=128,min_scale=0.75),3)

In [ ]:
arch = 'convnext_small_in22k'

learn = train(arch,Resize(192),aug_transforms(size=128,min_scale=0.75))

In [ ]:
def train(arch,size,item=Resize(480,method="squish"),accum=1,finetune= True,epochs = 12):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item,batch_tfms=aug_transforms(size=size,min_scale=0.75),bs=64//accum)
    cbs = GradientAccumulation(64) if accum  else []
    learn = vision_learner(dls, arch,metrics=error_rate,cbs=cbs).to_fp16()
    if(finetune):
        learn.fine_tune(epochs,0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs,0.01)

learn = train('convnext_small_in22k',128,epochs=1,accum=1,finetune=False)

In [ ]:
res = 640,480
models ={
    "convnext_large_in22k": {
        (Resize(res),(320,244))
    },
    'vit_large_match16_224':{
        (Resize(480,method="squish"),224),
        (Resize(res),224)
    },
    'swinv2_large_window12_192_22k':{
        (Resize(480,method="squish"),192),
        (Resize(res),192)
    },
    'swin_large_patch4_window7_224':{
        (Resize((res),224))
    }
}


In [ ]:
import gc 
trn_path = path/'train_images'

tta_res = []
for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch,size,item=item,accum=2))
        gc.collect()
        torch.cuda.empty_cache()
    
    